# Government Tech Transfer Raw Data - Sensitive

In [1]:
import pandas as pd
import re
import pandas as pd
import random
from datetime import datetime, timedelta

In [2]:
def extract_member_info(file_path):
    """
    Extract information from Excel spreadsheet with the (expected) structure in column A:
    Row 1: Name
    Row 2: Email
    Row 3: Blank
    Row 4: Job Title

    Organisation we want is in column D in the same row as the name.
    Areas of expertise are in column E for the rows from name to job title.

    No columns have been cleaned beyond just extraction.
    For the last few sets of names, the ordering was incorrect. Basing on using email as second row to name, but for some it was third row. Corrected for manually.
    """
    df = pd.read_excel(file_path)

    names = []
    emails = []
    job_titles = []
    organisations = []
    expertise_list = []
    help_list = []

    # Use while lopp to set up a repeating 4-row pattern rather than for loop, so system knows new name block after 4 rows.
    i = 0
    while i < len(df):
        # Skip header rows or empty rows
        if i < len(df) and (
            pd.isna(df.iloc[i, 0])
            or (isinstance(df.iloc[i, 0], str) and "Group member" in df.iloc[i, 0])
        ):
            i += 1
            continue

        # Check if current row might be a name (row 1 of the pattern)
        if i < len(df) and not pd.isna(df.iloc[i, 0]) and not pd.isna(df.iloc[i, 3]):
            # Expecting a consistent pattern - next row should not be empty, and contain an email, then blank, then more text. If so, then we start a new entry block
            if (
                i + 1 < len(df)
                and not pd.isna(df.iloc[i + 1, 0])
                and isinstance(df.iloc[i + 1, 0], str)
                and "@" in df.iloc[i + 1, 0]
                and i + 3
                < len(
                    df
                )  # Want at least current row + 3 for a complete name block, otherwise we reached the end and no more names.
                and not pd.isna(df.iloc[i + 3, 0])
            ):
                # Extract information from column A and C
                name = df.iloc[i, 0].strip()
                email = df.iloc[i + 1, 0].strip()
                job_title = df.iloc[i + 3, 0].strip()
                organisation = (
                    df.iloc[i, 3].strip() if not pd.isna(df.iloc[i, 3]) else ""
                )

                # Extract all areas of expertise from rows i to i+3 (name to job title)
                expertise = []
                for j in range(i, i + 4):
                    if j < len(df) and not pd.isna(df.iloc[j, 4]):
                        exp_text = str(df.iloc[j, 4])
                        # Minor tidy - remove bullets
                        exp_text = re.sub(r"^[-•\s]+", "", exp_text).strip()
                        if exp_text:
                            expertise.append(exp_text)

                # Extract all areas ofrequested help from rows i to i+3 (name to job title)
                help = []
                for j in range(i, i + 4):
                    if j < len(df) and not pd.isna(df.iloc[j, 4]):
                        help_text = str(df.iloc[j, 4])
                        # Minor tidy - remove bullets
                        help_text = re.sub(r"^[-•\s]+", "", help_text).strip()
                        if help_text:
                            help.append(exp_text)

                names.append(name)
                emails.append(email)
                job_titles.append(job_title)
                organisations.append(organisation)
                expertise_list.append("; ".join(expertise))
                help_list.append("; ".join(help))

                # If pattern matched, skip forward 4 rows, onto the next name
                i += 4
            else:
                # Pattern didn't match exactly, so move forward one instead and see if can find a pattern matc for the next row
                i += 1
        else:
            # Not a match for the start of a block, skip forward one when the row is empty or likely a header row
            i += 1

    # Create DataFrame from the extracted data
    result_df = pd.DataFrame(
        {
            "Name": names,
            "Email": emails,
            "Job Title": job_titles,
            "Organisation": organisations,
            "Areas of Expertise": expertise_list,
            "Requested help": help_list,
        }
    )

    return result_df

In [3]:
members = extract_member_info("Data/ivana_raw_data.xlsx")

In [8]:
def dummy_data_maker(extracted_df):
    # Create a mock structure for later
    structured_data = {
        # "id": [],
        # "parentId": [],
        "name": [],
        "email": [],
        "job_title": [],
        "grade": [],
        "location": [],
        # "team": [],
        # "area": [],
        # "unit": [],
        # "directorate": [],
        "department": [],
        "skills": [],
        "help": [],
    }

    # Use the previous csv file for some real values to skim from randomly
    grades = [
        "SCS4",
        "SCS3",
        "SCS2",
        "SCS1",
        "G6",
        "G7",
        "SEO",
        "HEO",
        "EO",
        "AO",
        "AA",
    ]
    locations = [
        "London",
        "Manchester",
        "Bristol",
        "Birmingham",
        "Leeds",
        "Glasgow",
        "Cardiff",
        "Belfast",
        "Newcastle",
        "Edinburgh",
    ]
    # teams = [
    #    "Digital Services",
    #    "Data Science",
    #    "Infrastructure",
    #    "Policy",
    #    "Operations",
    #    "Research",
    #    "Analysis",
    #    "Communications",
    #    "Strategy",
    #    "Design",
    #    "Finance",
    #    "HR",
    #    "Technology",
    #    "Cyber Security",
    # ]
    # areas = [
    #    "Digital Transformation",
    #    "Service Design",
    #    "User Research",
    #    "Data Analysis",
    #    "Software Development",
    #    "Cloud Infrastructure",
    #    "Cyber Security",
    #    "Policy Development",
    #    "Operations",
    # ]
    # units = [
    #    "Delivery",
    #    "Enabling Functions",
    #    "Strategy & Policy",
    #    "Technology",
    #    "People",
    #    "Finance",
    # ]
    # directorates = ["Digital", "Operations", "Strategy", "Technology", "Corporate"]

    # Helper function to select randomly or return empty with specified probability
    def random_choice(options, empty_prob=0.15):
        if random.random() < empty_prob:
            return ""
        else:
            return random.choice(options)

        # Generate experience descriptions based on skills, otherwise it can be empty
        # def generate_experience(skills, years):
        #    if not skills:
        #        return ""

        #    skill_parts = skills.split("/")[:4]
        #    if not skill_parts:
        #        return ""

        #    # Create some dummy experience text
        #    experience_templates = [
        #        f"{years} years experience in {', '.join(skill_parts)}. Led multiple initiatives in these areas.",
        #        f"Over {years} years specialising in {' and '.join(skill_parts)}. Extensive project delivery experience.",
        #        f"{years} years focusing on {skill_parts[0]} with additional expertise in related areas.",
        #        f"Career spanning {years} years with deep expertise in {', '.join(skill_parts)}. Known for innovative approaches.",
        #        f"{years} years building capabilities in {', '.join(skill_parts)}. Successfully delivered complex projects.",
        #    ]

        # return random.choice(experience_templates)

    print(extracted_df.head(1))

    for i, row in extracted_df.iterrows():
        # Real data fields
        name = row["Name"]
        email = (
            row["Email"]
            if pd.notna(row["Email"])
            else f"{name.lower().replace(' ', '.')}@dummy-gov.uk"
        )
        job_title = row["Job Title"] if pd.notna(row["Job Title"]) else ""
        department = row["Organisation"] if pd.notna(row["Organisation"]) else ""

        # Set up areas of expertise ready to become skills columns
        skills = (
            row["Areas of Expertise"] if pd.notna(row["Areas of Expertise"]) else ""
        )
        skills = skills.replace("; ", "/")

        help_required = row["Requested help"] if pd.notna(row["Requested help"]) else ""
        help_required = help_required.replace("; ", "/")

        # Dummy fields with ~5-15% empty rate to simulate people not bothering with adding information
        # record_id = i
        # parent_id = (
        #    "" if random.random() > 0.2 else str(random.randint(0, max(0, i - 1)))
        # )
        grade = random_choice(grades, 0.10)  # Only 10% empty grades
        location = random_choice(locations, 0.15)  # Only 15% empty locations
        # team = random_choice(
        #    teams, 0.05
        # )  # 5% empty on assumption maybe we can take this from HR
        # area = random_choice(areas, 0.08)  # 8% empty
        # unit = random_choice(units, 0.05)  # 5% empty
        # directorate = random_choice(directorates, 0.5)  # 5% empty

        # Generate experience text based on skills and random years of experience
        # years_of_experience = random.randint(3, 25)
        # experience = generate_experience(skills, years_of_experience)

        # Add to structured data
        # structured_data["id"].append(record_id)
        # structured_data["parentId"].append(parent_id)
        structured_data["name"].append(name)
        structured_data["email"].append(email)
        structured_data["job_title"].append(job_title)
        structured_data["grade"].append(grade)
        structured_data["location"].append(location)
        # structured_data["team"].append(team)
        # structured_data["area"].append(area)
        # structured_data["unit"].append(unit)
        # structured_data["directorate"].append(directorate)
        structured_data["department"].append(department)
        structured_data["skills"].append(skills)
        structured_data["help"].append(help_required)
        # structured_data["experience"].append(experience)

    # Create DataFrame
    result_df = pd.DataFrame(structured_data)
    return result_df

In [ ]:
dummy_data = dummy_data_maker(members)

In [10]:
# Write to CSV ready for use later
dummy_data.to_csv("techtransfer_raw.csv")

In [7]:
#